In [1]:
# from pyimagesearch.localbinarypatterns import LocalBinaryPatterns
# from sklearn.svm import LinearSVC
# from imutils import paths
# import argparse
# import cv2
# import os
# import numpy as np
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
from matplotlib import pyplot as plt
import numpy as np
import argparse
import imutils
import cv2
import os
%matplotlib inline

In [2]:
def create_gaborfilter():
    # This function is designed to produce a set of GaborFilters 
    # an even distribution of theta values equally distributed amongst pi rad / 180 degree
     
    filters = []
    num_filters = 16
    ksize = 35  # The local area to evaluate
    sigma = 3.0  # Larger Values produce more edges
    lambd = 10.0
    gamma = 0.5
    psi = 0  # Offset value - lower generates cleaner results
    for theta in np.arange(0, np.pi, np.pi / num_filters):  # Theta is the orientation for edge detection
        kern = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_64F)
        kern /= 1.0 * kern.sum()  # Brightness normalization
        filters.append(kern)
    return filters

In [3]:
# Fungsi untuk membuat kernel Gabor
def gabor_kernel(size, theta, lambd, sigma, gamma):
    kernel = cv2.getGaborKernel((size, size), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_32F)
    return kernel

# Fungsi untuk menerapkan Gabor filter ke citra
def apply_gabor_filter(image, kernels):
    filtered_images = [cv2.filter2D(image, cv2.CV_8UC3, k) for k in kernels]
    return filtered_images

# Fungsi untuk mengambil fitur dari gambar hasil Gabor filter
def extract_features_from_gabor_images(gabor_images):
    features = [image.flatten() for image in gabor_images]
    return features

In [4]:
def apply_filter(img, filters):
# This general function is designed to apply filters to our image
     
    # First create a numpy array the same size as our input image
    newimage = np.zeros_like(img)
     
    # Starting with a blank image, we loop through the images and apply our Gabor Filter
    # On each iteration, we take the highest value (super impose), until we have the max value across all filters
    # The final image is returned
    depth = -1 # remain depth same as original image
     
    for kern in filters:  # Loop through the kernels in our GaborFilter
        image_filter = cv2.filter2D(img, depth, kern)  #Apply filter to image
         
        # Using Numpy.maximum to compare our filter and cumulative image, taking the higher value (max)
        np.maximum(newimage, image_filter, newimage)
    return newimage

In [5]:
def image_to_feature_vector(image, size=(30, 30)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [6]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	cv2.normalize(hist, hist)
	return hist.flatten()

In [7]:
#for plt stuff
def showimage(myimage, figsize=[10,10]):
    if (myimage.ndim>2):  #This only applies to RGB or RGBA images (e.g. not to Black and White images)
        myimage = myimage[:,:,::-1] #OpenCV follows BGR order, while matplotlib likely follows RGB order
         
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(myimage, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

In [8]:
# the data and label lists
#desc = LocalBinaryPatterns(24, 8)
data = []
print("[INFO] describing images...")
train_dir = "/home/hifzhil/myWork/DSEC/Dataset/dataset"


#labels = []

[INFO] describing images...


In [9]:
#for raw image (natural image)
rawImages = []

#for histogram feature KNN
features = []

#for texture extraction data
textures = []

#for label
labels = []

#texture extraction using gabor
gfilters = create_gaborfilter()
size = 11
theta = 0
lambd = 5.0
sigma = 1.0
gamma = 0.02
i = 0

gabor_kernels = [gabor_kernel(size, theta, lambd, sigma, gamma * i) for i in range(1)]

for (imagePath) in paths.list_images(train_dir):
    # load the image, convert it to grayscale, and describe it
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #hist = desc.describe(gray)
    
    #for raw data
    pixels = image_to_feature_vector(image)
    
    #for histogram data
    hist = extract_color_histogram(image)

    #for texture data
    image_g = apply_filter(image, gfilters)
    image_gb = image_to_feature_vector(image_g)
    # gabor_kernels = [gabor_kernel(size, theta, lambd, sigma, gamma * i) for i in range(4)]

    # # Terapkan Gabor filter ke citra
    # filtered_images = apply_gabor_filter(image, gabor_kernels)

    # # Ekstrak fitur dari gambar hasil Gabor filter
    # feature_vector = extract_features_from_gabor_images(filtered_images)

    
# extract the label from the image path, then update the
    # label and data lists
    labels.append(imagePath.split(os.path.sep)[-2])
    #data.append(hist)
    rawImages.append(pixels)
    features.append(hist)
    textures.append(image_gb)
    i+=1
    if i > 0 and i % 2 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePath)))
# # train a Linear SVM on the data
# model = LinearSVC(C=100.0, random_state=42)
# model.fit(data, labels)

[INFO] processed 2/57
[INFO] processed 4/53
[INFO] processed 6/53
[INFO] processed 8/51
[INFO] processed 10/54
[INFO] processed 12/54
[INFO] processed 14/57
[INFO] processed 16/59
[INFO] processed 18/59
[INFO] processed 20/61
[INFO] processed 22/57
[INFO] processed 24/57
[INFO] processed 26/59
[INFO] processed 28/61
[INFO] processed 30/61
[INFO] processed 32/59


In [10]:
print(labels)

['penghapus', 'penghapus', 'penghapus', 'panci', 'panci', 'panci', 'oli', 'oli', 'oli', 'liquid', 'liquid', 'liquid', 'teh_celup', 'teh_celup', 'teh_celup', 'tolak_angin', 'tolak_angin', 'tolak_angin', 'gelas_plastik', 'gelas_plastik', 'gelas_plastik', 'jam_weker', 'jam_weker', 'jam_weker', 'botol_putih', 'botol_putih', 'botol_putih', 'jangka_sorong', 'jangka_sorong', 'jangka_sorong', 'teko_heater', 'teko_heater', 'teko_heater']


In [11]:
rawImages = np.array(rawImages)
features = np.array(features)
textures = np.array(textures)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(
	rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(
	features.nbytes / (1024 * 1000.0)))
print("[INFO] textures matrix: {:.2f}MB".format(
	textures.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 0.09MB
[INFO] features matrix: 0.07MB
[INFO] textures matrix: 0.09MB


In [12]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)
(trainTI, testTI, trainTL, testTL) = train_test_split(
	textures, labels, test_size=0.25, random_state=42)

In [13]:
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model_raw = KNeighborsClassifier(n_neighbors=3,
	n_jobs=-1)
model_raw.fit(trainRI, trainRL)
acc = model_raw.score(testRI, testRL)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 66.67%


In [15]:
# train and evaluate a k-NN classifer on the histogram representations
print("[INFO] evaluating histogram accuracy...")
model_hist = KNeighborsClassifier(n_neighbors=3,
	n_jobs=-1)
model_hist.fit(trainFeat, trainLabels)
predic_now = model_hist.predict(trainFeat)
acc = model_hist.score(testFeat, testLabels)
print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))
#print("[INFO] Prediction : {:.2f}%".format(predic_now))

[INFO] evaluating histogram accuracy...
[INFO] histogram accuracy: 66.67%


In [16]:
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating textures accuracy...")
model_texture = KNeighborsClassifier(n_neighbors=3,
	n_jobs=-1)
model_texture.fit(trainTI, trainTL)
acc = model_texture.score(testTI, testTL)
print("[INFO] textures accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating textures accuracy...
[INFO] textures accuracy: 55.56%


In [24]:
what_model_you_need = input('what model you need ? \n (1) Pure / Raw  \n (2) Hist \n (3) Texture \n')

what model you need ? 
 (1) Pure / Raw  
 (2) Hist 
 (3) Texture 
 3


In [25]:
what_you_looking = input('what object should search for?')

what object should search for? oli


In [ ]:
for imagePath in paths.list_images(train_dir):
  # load the image, convert it to grayscale, describe it,
  # and classify it 
  image = cv2.imread(imagePath)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #hist = desc.describe(gray)
  hist_image = extract_color_histogram(image)
  test_image = image_to_feature_vector(image)
  if(what_model_you_need == "1") :
      model = model_raw
  elif(what_model_you_need == "2") :
      model = model_hist
  elif(what_model_you_need == "3") :
      model = model_texture
  ### don't forget to make it as numpy before testing
  prediction = model.predict(np.array([test_image]))
  # display the image and the prediction
  scale_percent = 50
  scaled_factor = (int(image.shape[1] * scale_percent / 100), int(image.shape[0] * scale_percent / 100))
  image_show = cv2.resize(image, scaled_factor, interpolation=cv2.INTER_AREA)
  print(prediction[0])
  if(what_you_looking == prediction[0]) :
      cv2.putText(image, prediction[0], (10, 30), cv2.FONT_HERSHEY_SIMPLEX,1.0, (0, 0, 255), 3)
      cv2.imshow("Image", image_show)
      cv2.waitKey(0)
      if cv2.waitKey(30) & 0xFF == 27:  # Tekan 'Esc' untuk keluar
          break
cv2.destroyAllWindows()

penghapus
penghapus
penghapus
panci
panci
gelas_plastik
liquid
oli
